## Vi kan introdusere uregelmessigheter til dataen ved diverse funksjoner

Dataen vi har samlet er nokså fullstendig, og uten feilverdier, men ved bruk av enkelte funksjoner kan vi introdusere noen feil, for så å rette de opp igjen etterpå: 

In [1]:
import pandas as pd
import numpy as np

df2_errors = pd.read_json("../data/df2_data.json") #leser json filen til "df2"

# introduserer uregelmessigheter (NaN til omtrent 6% av verdiene)
def introduser_NaN(entry):
    entry.loc[entry.sample(frac=0.06).index, 'mean(relative_humidity P1D)'] = np.nan 
    return entry

# introduserer enkelte tallverdier på 570, som ikke gir mening (uteliggere)
def introduser_uteliggere(entry):
    entry.loc[entry.sample(frac=0.04).index, 'mean(air_temperature P1D)'] = 570
    return df2_errors

#introduserer "error" i boolean feltene
def introduser_non_boolean(entry):
    entry.loc[entry.sample(frac=0.04).index,'boolean_clear_sky_weather(cloud_area_fraction P1D)']= "error"
    return df2_errors

#Kjører funksjonene og lagrer til ny fil
nan=introduser_NaN(df2_errors)
uteligger=introduser_uteliggere(nan)
final_error_file=introduser_non_boolean(uteligger)


final_error_file.to_json('../data/df2_errors.json',indent=4)







/var/folders/v8/s6vl_7bn6fd74pxnx3tdftb80000gn/T/ipykernel_53723/2017829464.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'error' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  entry.loc[entry.sample(frac=0.04).index,'boolean_clear_sky_weather(cloud_area_fraction P1D)']= "error"
